<a href="https://colab.research.google.com/github/httpsb/machine-learning/blob/main/ML2_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torchvision import datasets

trainset = datasets.MNIST(root='./data', train=True, download=True)
testset = datasets.MNIST(root='./data', train=False, download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 35682221.46it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1118478.48it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9727826.71it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4866035.45it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
np.random.seed(0)
val_ratio = 0.1
train_size = len(trainset)
indices = list(range(train_size))
split_idx = int(np.floor(val_ratio * train_size))
np.random.shuffle(indices)
train_idx, val_idx = indices[split_idx:], indices[:split_idx]

In [ ]:
train_data = trainset.data[train_idx].float()/255
train_labels = trainset.targets[train_idx]
val_data = trainset.data[val_idx].float()/255
val_labels = trainset.targets[val_idx]
test_data = testset.data.float()/255
test_labels = testset.targets

In [ ]:
print('shape of train_data is ', train_data.shape)
print('shape of train_labels is ', train_labels.shape)
print('shape of val_data is ', val_data.shape)
print('shape of val_labels is ', val_labels.shape)
print('shape of test_data is ', test_data.shape)
print('shape of test_labels is ', test_labels.shape)

shape of train_data is  torch.Size([54000, 28, 28])
shape of train_labels is  torch.Size([54000])
shape of val_data is  torch.Size([6000, 28, 28])
shape of val_labels is  torch.Size([6000])
shape of test_data is  torch.Size([10000, 28, 28])
shape of test_labels is  torch.Size([10000])


In [ ]:
#check if cuda is available
print(torch.cuda.is_available())
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device")

True
using cuda device


In [ ]:
#move all necessary data to GPU
train_data = train_data.to("cuda")
train_labels = train_labels.to("cuda")
val_data = val_data.to("cuda")
val_labels = val_labels.to("cuda")
test_data = test_data.to("cuda")
test_labels = test_labels.to("cuda")

In [ ]:
#(a)

#calculate distance
def euclidean_distance(x1, x2):
  return torch.sqrt(torch.sum((x1-x2)**2))

def knn_predict(new_example, train_data, train_labels, k=5):

  #calculate distance from new example to all training distance
  distances = []
  for i in range(len(train_data)):
    distance = euclidean_distance(new_example, train_data[i])
    distances.append((distance.item(), i))

  #sort distance in ascending order and select k
  distances.sort(key=lambda x:x[0])
  k_nearest_neighbours = distances[:k]

  #extract labels of nearest neighbors
  k_nearest_labels = [train_labels[i] for _, i in k_nearest_neighbours]

  #voting to get most common label
  majority = torch.mode(torch.tensor(k_nearest_labels)).values
  return majority


In [ ]:
import time
start_time = time.time()

new_example = test_data[0]

predicted_label = knn_predict(new_example, train_data, train_labels, k=5)
print(f"Predicted label: {predicted_label}")

true_label = test_labels[0]
print(f"True label: {true_label}")

end_time = time.time()
print(f"Execution time is: {end_time-start_time}")


Predicted label: 7
True label: 7
Execution time is: 1.4906089305877686


In [ ]:
#(b)
#classify a new example using knn with broadcasting
def knn_predict_broadcast(new_example, train_data, train_labels, k=5):
  #calculate distance
  #euclidean dist
  distances = torch.sqrt(torch.sum((train_data - new_example)**2, dim =(1,2)))

  #get indices of k
  k_distance, k_indices = torch.topk(distances, k, largest=False)

  #get labels of k
  k_nearest_labels = train_labels[k_indices]

  #return most common class label
  return torch.argmax(torch.bincount(k_nearest_labels))

In [ ]:
start_time = time.time()

new_example = test_data[0]
predicted_label_broadcast = knn_predict_broadcast(new_example, train_data, train_labels, k=5)

print(f"Predicted label using broadcasting: {predicted_label_broadcast}")

true_label_broadcast = test_labels[0]
print(f"True label: {true_label_broadcast}")

end_time = time.time()
print(f"Execution time is: {end_time-start_time}")

Predicted label using broadcasting: 7
True label: 7
Execution time is: 0.28426146507263184


In [ ]:
#(c) extend from b
#classify a new example using knn with broadcasting
def knn_predict_c(new_example, train_data, train_labels, k=5):
  #calculate distance
  #euclidean dist
  distances = torch.sqrt(torch.sum((train_data.unsqueeze(0) - new_example.unsqueeze(1))**2, dim =(2,3)))

  #get indices of k
  k_distance, k_indices = torch.topk(distances, k, largest=False)

  #get labels of k
  k_nearest_labels = train_labels[k_indices]

  #return most common class label
  return torch.argmax(torch.bincount(k_nearest_labels))

In [ ]:
new_example = test_data
pred_label_c = knn_predict_c(new_example, train_data, train_labels, k=5)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1577.14 GiB. GPU 0 has a total capacity of 14.75 GiB of which 14.44 GiB is free. Process 9025 has 312.00 MiB memory in use. Of the allocated memory 210.53 MiB is allocated by PyTorch, and 1.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#question (c) and (d)

def knn_predict_bybatch(test_data, train_data, train_labels, k, p, batch_size=1000):
    all_pred = []  # To store all predictions
    num_test_sample = test_data.shape[0]

    with torch.no_grad():
        for start in range(0, num_test_sample, batch_size):
            end = min(start + batch_size, num_test_sample)
            big_chunk = test_data[start:end]

            # Reshape for distance calculation
            big_chunk_flat = big_chunk.view(-1, 28*28)   # [1000, 28*28]
            train_data_flat = train_data.view(-1, 28*28)  # [54000, 28*28]

            torch.cuda.empty_cache()

            # Calculate pairwise distances using torch.cdist
            distance = torch.cdist(big_chunk_flat, train_data_flat, p=p)

            # Get indices of k nearest neighbors
            k_indices = torch.topk(distance, k, largest=False).indices

            # Get labels of k nearest neighbors
            k_nearest_labels = train_labels[k_indices]

            # Return most common class label for current size
            pred, _ = torch.mode(k_nearest_labels, dim=1) #using mode instead of bincount
            all_pred.append(pred)  # Append to all

    return torch.cat(all_pred)  # Return all predictions (concatenate into one tensor)

In [ ]:
#do classification on all digits in test_data
pred_label_batch_p2 = knn_predict_bybatch(test_data, train_data, train_labels, k=5, p=2)
k5_p2_test_acc = (pred_label_batch_p2 == test_labels).float().mean()
print(f"test accuracy when k=5 and p=2 is: {k5_p2_test_acc*100:.2f}% ")

pred_label_val = knn_predict_bybatch(val_data, train_data, train_labels, k=5, p=2)
k5_p2_val_acc = (pred_label_val == val_labels).float().mean()
print(f'val accuracy when k=5 and p=2 is {k5_p2_val_acc*100:.2f}%')

test accuracy when k=5 and p=2 is: 96.65% 
val accuracy when k=5 and p=2 is 97.37%


In [ ]:
size_arr = [5, 10, 15, 50, 100, 1000]

for n in size_arr:
  pred_label_batch = knn_predict_bybatch(val_data, train_data, train_labels, k=5, p=2, batch_size=n)
  val_accuracy = (pred_label_batch == val_labels).float().mean()
  print(f'Validation accuracy for batch_size {n}: {val_accuracy.item() * 100:.2f}%')

Validation accuracy for batch_size 5: 97.37%
Validation accuracy for batch_size 10: 97.37%
Validation accuracy for batch_size 15: 97.37%
Validation accuracy for batch_size 50: 97.37%
Validation accuracy for batch_size 100: 97.37%
Validation accuracy for batch_size 1000: 97.37%


Since batch size doesnt seem to improve accuracy, we can try tuning other hyperparameter. In this case, I chose distance calculation.


In [ ]:
#now check for best values of k and which distance calculation is the best, euclidean or manhattan
arr_k = [1,3,5,10,13,18]
p_arr = [1,2]
val_acc = []
best_acc = 0
best_k = None
best_p = None

for k in arr_k:
    for p in p_arr:
        pred_label_batch = knn_predict_bybatch(val_data, train_data, train_labels, k, p)
        val_accuracy = (pred_label_batch == val_labels).float().mean()
        print(f'Validation accuracy for k={k}, p: {p}: {val_accuracy.item() * 100:.2f}%')
        val_acc.append(val_accuracy.item())

        if best_acc < val_accuracy.item():
          best_acc = val_accuracy.item()
          best_k = k
          best_p = p

print(f"Best accuracy is {best_acc*100:.2f}% where the values of k is {best_k} and p is {best_p}")

Validation accuracy for k=1, p: 1: 96.77%
Validation accuracy for k=1, p: 2: 97.47%
Validation accuracy for k=3, p: 1: 96.93%
Validation accuracy for k=3, p: 2: 97.55%
Validation accuracy for k=5, p: 1: 96.55%
Validation accuracy for k=5, p: 2: 97.37%
Validation accuracy for k=10, p: 1: 96.50%
Validation accuracy for k=10, p: 2: 97.17%
Validation accuracy for k=13, p: 1: 96.23%
Validation accuracy for k=13, p: 2: 96.88%
Validation accuracy for k=18, p: 1: 96.10%
Validation accuracy for k=18, p: 2: 96.75%
Best accuracy is 97.55% where the values of k is 3 and p is 2


In [ ]:
#val data accuracy
pred_label_batch_val = knn_predict_bybatch(val_data, train_data, train_labels, k=3, p=2)

val_accuracy = (pred_label_batch_val == val_labels).float().mean()
print(f'val test accuracy: {val_accuracy.item() * 100:.2f}%')#do classification on all digits in test_data
pred_label_batch_test = knn_predict_bybatch(test_data, train_data, train_labels, k=3, p=2)

#final test accuracy
pred_label_batch_test = knn_predict_bybatch(test_data, train_data, train_labels, k=3, p=2)

test_accuracy = (pred_label_batch_test == test_labels).float().mean()
print(f'test accuracy: {test_accuracy.item() * 100:.2f}%')

val test accuracy: 97.55%
test accuracy: 96.97%
